In [ ]:
!pip install unsloth -q

In [ ]:
from unsloth import FastVisionModel
import torch

model, processor = FastVisionModel.from_pretrained(
    "google/gemma-3-4b-it",
    load_in_4bit = False,
    use_gradient_checkpointing = "unsloth",
)

In [ ]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,

    r = 16,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
    target_modules = "all-linear",
)

In [ ]:
!pip install datasets -q -U

In [ ]:
from datasets import load_dataset
dataset = load_dataset("IRI2070/shenas-verified-bookroom-persian-book-covers-and-titles")

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'text', 'is_valid'],
        num_rows: 7251
    })
    test: Dataset({
        features: ['image', 'text', 'is_valid'],
        num_rows: 5198
    })
})

In [7]:
test_samples = dataset['test'].select(range(200))
train_samples = dataset['train'].shuffle(seed=42).select(range(1000))

In [8]:
import PIL.Image
from io import BytesIO
from datasets import Dataset

instruction = "Extract only the Persian title of the book exactly as it appears on the cover in the image. The output must be **solely** the Persian title in a single line, with nothing else added—no Latin transliteration, no English translation, no explanations, no additional text, and no extra lines. Do not write anything except the Persian title itself."

def convert_to_conversation(sample):
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": instruction},
                {"type": "image", "image": sample["image"]},
            ],
        },
        {"role": "assistant", "content": [{"type": "text", "text": sample["text"]}]},
    ]
    return {"messages": conversation}

converted_dataset = [convert_to_conversation(sample) for sample in train_samples]
converted_test_samples = [convert_to_conversation(sample) for sample in test_samples]

In [9]:
from unsloth import get_chat_template

processor = get_chat_template(
    processor,
    "gemma-3"
)

In [ ]:
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model)

trainer = SFTTrainer(
    model=model,
    train_dataset=converted_dataset,
    processing_class=processor.tokenizer,
    data_collator=UnslothVisionDataCollator(model, processor),
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        gradient_checkpointing = True,

        gradient_checkpointing_kwargs = {"use_reentrant": False},
        max_grad_norm = 0.3,
        warmup_ratio = 0.03,
        # max_steps = 100,
        num_train_epochs = 3,
        learning_rate = 2e-4,
        logging_steps = 1,
        save_strategy="steps",
        optim = "adamw_torch_fused",
        weight_decay = 0.001,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",

        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        max_length = 2048,
    )
)

In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 3 | Total steps = 750
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 38,497,792 of 4,338,577,264 (0.89% trained)


Step,Training Loss
1,6.193800
2,5.930100
3,5.794800
4,5.810700
5,5.279700
6,4.581500
7,4.552600
8,3.687000
9,3.786600
10,3.149700
